<a href="https://colab.research.google.com/github/thomasshin/NLP_Study/blob/main/HuggingFace_NLP_Course/Tokenization_Algorithm/BPE_Tokenization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 89.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.7 MB/s eta 0:00:00


#Byte-Pair Encoding tokenization

Byte-Pair Encoding (BPE) was initially developed as an algorithm to compress texts, and then used by OpenAI for tokenization when pretraining the GPT model. It’s used by a lot of Transformer models, including GPT, GPT-2, RoBERTa, BART, and DeBERTa.

#Implementing BPE

Now let’s take a look at an implementation of the BPE algorithm. This won’t be an optimized version you can actually use on a big corpus; we just want to show you the code so you can understand the algorithm a little bit better.

First we need a corpus, so let’s create a simple one with a few sentences:

In [2]:
corpus = [
    "This is the Hugging Face Course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",
]

Next, we need to pre-tokenize that corpus into words. Since we are replicating a BPE tokenizer (like GPT-2), we will use the gpt2 tokenizer for the pre-tokenization:

In [3]:
from transformers import AutoTokenizer

tokenizer_gpt2 = AutoTokenizer.from_pretrained("gpt2")

In [4]:
for text in corpus:
  print(tokenizer_gpt2.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text))

[('This', (0, 4)), ('Ġis', (4, 7)), ('Ġthe', (7, 11)), ('ĠHugging', (11, 19)), ('ĠFace', (19, 24)), ('ĠCourse', (24, 31)), ('.', (31, 32))]
[('This', (0, 4)), ('Ġchapter', (4, 12)), ('Ġis', (12, 15)), ('Ġabout', (15, 21)), ('Ġtokenization', (21, 34)), ('.', (34, 35))]
[('This', (0, 4)), ('Ġsection', (4, 12)), ('Ġshows', (12, 18)), ('Ġseveral', (18, 26)), ('Ġtokenizer', (26, 36)), ('Ġalgorithms', (36, 47)), ('.', (47, 48))]
[('Hopefully', (0, 9)), (',', (9, 10)), ('Ġyou', (10, 14)), ('Ġwill', (14, 19)), ('Ġbe', (19, 22)), ('Ġable', (22, 27)), ('Ġto', (27, 30)), ('Ġunderstand', (30, 41)), ('Ġhow', (41, 45)), ('Ġthey', (45, 50)), ('Ġare', (50, 54)), ('Ġtrained', (54, 62)), ('Ġand', (62, 66)), ('Ġgenerate', (66, 75)), ('Ġtokens', (75, 82)), ('.', (82, 83))]


Then we compute the frequencies of each word in the corpus as we do the pre-tokenization:

In [5]:
from collections import defaultdict

word_freqs = defaultdict(int)

for text in corpus:
  words_with_offsets = tokenizer_gpt2.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
  new_words = [word for word, offsets in words_with_offsets]
  for word in new_words:
    word_freqs[word] += 1

In [6]:
word_freqs

defaultdict(int,
            {'This': 3,
             'Ġis': 2,
             'Ġthe': 1,
             'ĠHugging': 1,
             'ĠFace': 1,
             'ĠCourse': 1,
             '.': 4,
             'Ġchapter': 1,
             'Ġabout': 1,
             'Ġtokenization': 1,
             'Ġsection': 1,
             'Ġshows': 1,
             'Ġseveral': 1,
             'Ġtokenizer': 1,
             'Ġalgorithms': 1,
             'Hopefully': 1,
             ',': 1,
             'Ġyou': 1,
             'Ġwill': 1,
             'Ġbe': 1,
             'Ġable': 1,
             'Ġto': 1,
             'Ġunderstand': 1,
             'Ġhow': 1,
             'Ġthey': 1,
             'Ġare': 1,
             'Ġtrained': 1,
             'Ġand': 1,
             'Ġgenerate': 1,
             'Ġtokens': 1})

The next step is to compute the base vocabulary, formed by all the characters used in the corpus:

In [7]:
alphabet = []

for word in word_freqs.keys():
  for i in word:
    if i not in alphabet:
      alphabet.append(i)
alphabet.sort()
alphabet

[',',
 '.',
 'C',
 'F',
 'H',
 'T',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'y',
 'z',
 'Ġ']

We also add the special tokens used by the model at the beginning of that vocabulary. In the case of GPT-2, the only special token is "<|endoftext|>":

In [8]:
vocab = ["<|endoftext|>"] + alphabet.copy()
vocab

['<|endoftext|>',
 ',',
 '.',
 'C',
 'F',
 'H',
 'T',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'y',
 'z',
 'Ġ']

We now need to split each word into individual characters, to be able to start training:

In [9]:
splits = {word:[i for i in word] for word in word_freqs.keys()}
splits

{'This': ['T', 'h', 'i', 's'],
 'Ġis': ['Ġ', 'i', 's'],
 'Ġthe': ['Ġ', 't', 'h', 'e'],
 'ĠHugging': ['Ġ', 'H', 'u', 'g', 'g', 'i', 'n', 'g'],
 'ĠFace': ['Ġ', 'F', 'a', 'c', 'e'],
 'ĠCourse': ['Ġ', 'C', 'o', 'u', 'r', 's', 'e'],
 '.': ['.'],
 'Ġchapter': ['Ġ', 'c', 'h', 'a', 'p', 't', 'e', 'r'],
 'Ġabout': ['Ġ', 'a', 'b', 'o', 'u', 't'],
 'Ġtokenization': ['Ġ',
  't',
  'o',
  'k',
  'e',
  'n',
  'i',
  'z',
  'a',
  't',
  'i',
  'o',
  'n'],
 'Ġsection': ['Ġ', 's', 'e', 'c', 't', 'i', 'o', 'n'],
 'Ġshows': ['Ġ', 's', 'h', 'o', 'w', 's'],
 'Ġseveral': ['Ġ', 's', 'e', 'v', 'e', 'r', 'a', 'l'],
 'Ġtokenizer': ['Ġ', 't', 'o', 'k', 'e', 'n', 'i', 'z', 'e', 'r'],
 'Ġalgorithms': ['Ġ', 'a', 'l', 'g', 'o', 'r', 'i', 't', 'h', 'm', 's'],
 'Hopefully': ['H', 'o', 'p', 'e', 'f', 'u', 'l', 'l', 'y'],
 ',': [','],
 'Ġyou': ['Ġ', 'y', 'o', 'u'],
 'Ġwill': ['Ġ', 'w', 'i', 'l', 'l'],
 'Ġbe': ['Ġ', 'b', 'e'],
 'Ġable': ['Ġ', 'a', 'b', 'l', 'e'],
 'Ġto': ['Ġ', 't', 'o'],
 'Ġunderstand': ['Ġ', 'u', 'n'

Now that we are ready for training, let’s write a function that computes the frequency of each pair. We’ll need to use this at each step of the training:

In [10]:
def compute_pair_freq(splits):
  pair_freqs = defaultdict(int)
  for word, freq in word_freqs.items():
    split = splits[word]
    if len(split) != 1:
      for i in range(len(split)-1):
        pair = (split[i], split[i+1])
        pair_freqs[pair] += 1
    else:
      continue
  return pair_freqs

In [11]:
pair_freqs = compute_pair_freq(splits)
for i, key in enumerate(pair_freqs.keys()):
    print(f"{key}: {pair_freqs[key]}")
    if i >= 5:
        break

('T', 'h'): 1
('h', 'i'): 1
('i', 's'): 2
('Ġ', 'i'): 1
('Ġ', 't'): 7
('t', 'h'): 3


Now, finding the most frequent pair only takes a quick loop:

In [12]:
best_pair = ""
max_freq = None

for pair, freq in pair_freqs.items():
    if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq

print(best_pair, max_freq)

('Ġ', 't') 7


So the first merge to learn is ('Ġ', 't') -> 'Ġt', and we add 'Ġt' to the vocabulary:

In [13]:
merges = {('Ġ', 't') : "Ġt"}
vocab.append("Ġt")

To continue, we need to apply that merge in our splits dictionary. Let’s write another function for this:

In [14]:
def merge_pair(a, b, splits):
  for word in word_freqs:
    split = splits[word]
    if len(split) == 1:
      continue

    i = 0
    while i < len(split)-1:
        if split[i] == a and split[i+1] == b:
          split = split[:i] + [a+b] + split[i+2:]
        else:
          i+=1
    splits[word] = split
  return splits

In [15]:
def merge_pair(a, b, splits):
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue

        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i + 1] == b:
                split = split[:i] + [a + b] + split[i + 2 :]
            else:
                i += 1
        splits[word] = split
    return splits

In [16]:
splits = merge_pair("Ġ", "t", splits)
print(splits["Ġtrained"])

['Ġt', 'r', 'a', 'i', 'n', 'e', 'd']


Now we have everything we need to loop until we have learned all the merges we want. Let’s aim for a vocab size of 50:

In [17]:
vocab_size = 50

while len(vocab) < vocab_size:
  pair_freqs = compute_pair_freq(splits)
  best_pair = ""
  max = None
  for pair, freqs in pair_freqs.items():
    if max == None or max < freqs:
      max = freqs
      best_pair = pair
  splits = merge_pair(*best_pair, splits)
  merges[best_pair] = best_pair[0] + best_pair[1]
  vocab.append(best_pair[0] + best_pair[1])

In [18]:
vocab_size = 50

while len(vocab) < vocab_size:
    pair_freqs = compute_pair_freq(splits)
    best_pair = ""
    max_freq = None
    for pair, freq in pair_freqs.items():
        if max_freq is None or max_freq < freq:
            best_pair = pair
            max_freq = freq
    splits = merge_pair(*best_pair, splits)
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0] + best_pair[1])

In [19]:
merges, vocab

({('Ġ', 't'): 'Ġt',
  ('e', 'r'): 'er',
  ('Ġ', 'a'): 'Ġa',
  ('Ġt', 'o'): 'Ġto',
  ('e', 'n'): 'en',
  ('o', 'u'): 'ou',
  ('s', 'e'): 'se',
  ('Ġto', 'k'): 'Ġtok',
  ('Ġtok', 'en'): 'Ġtoken',
  ('n', 'd'): 'nd',
  ('i', 's'): 'is',
  ('Ġt', 'h'): 'Ġth',
  ('Ġth', 'e'): 'Ġthe',
  ('i', 'n'): 'in',
  ('Ġa', 'b'): 'Ġab',
  ('Ġtoken', 'i'): 'Ġtokeni',
  ('Ġtokeni', 'z'): 'Ġtokeniz',
  ('a', 't'): 'at',
  ('i', 'o'): 'io'},
 ['<|endoftext|>',
  ',',
  '.',
  'C',
  'F',
  'H',
  'T',
  'a',
  'b',
  'c',
  'd',
  'e',
  'f',
  'g',
  'h',
  'i',
  'k',
  'l',
  'm',
  'n',
  'o',
  'p',
  'r',
  's',
  't',
  'u',
  'v',
  'w',
  'y',
  'z',
  'Ġ',
  'Ġt',
  'er',
  'Ġa',
  'Ġto',
  'en',
  'ou',
  'se',
  'Ġtok',
  'Ġtoken',
  'nd',
  'is',
  'Ġth',
  'Ġthe',
  'in',
  'Ġab',
  'Ġtokeni',
  'Ġtokeniz',
  'at',
  'io'])

To tokenize a new text, we pre-tokenize it, split it, then apply all the merge rules learned:

In [20]:
def tokenize(text):
  pre_tokenize_result = tokenizer_gpt2.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
  pre_tokenize_text = [word for word, offset in pre_tokenize_result]
  splits = [[l for l in word] for word in pre_tokenize_text]
  for pair, merge in merges.items():
    for idx, split in enumerate(splits):
      i = 0
      while i < len(split) - 1:
        if split[i] == pair[0] and split[i + 1] == pair[1]:
          split = split[:i] + [merge] + split[i + 2 :]
        else:
          i += 1
      splits[idx] = split

  return sum(splits, [])

In [21]:
tokenize("This is not a token.")

['T', 'h', 'is', 'Ġ', 'is', 'Ġ', 'n', 'o', 't', 'Ġa', 'Ġtoken', '.']